# Debugging

Notebooke to debug some issues in data files and functions in module

## Network annotation

In [8]:
import os, sys
os.chdir(os.path.expanduser('~/capstone-project'))
sys.path.append('src')

import networkx as nx
import pandas as pd

import cluster_analysis
import gene_associations
import network_construction
import network_analysis
import cluster_analysis
import data_analysis

species='Escherichia_coli'; drug="trimethoprim"

presence_path = f'data/presence_matrices/{species}_filtered_GxS.csv' #needed later on for resistance lors
presence_df = pd.read_csv(presence_path, index_col=0)
pheno_path = f'data/processed_phenotypes/{species}_{drug}.csv' 

#  ----------------- cluster info -----------------
cluster_info_df = pd.read_csv(f'data/clusters/{species}_cluster_info_filtered.csv', index_col=0)

# ----------------- LOR of resistance -----------------
lor_resistance_scores = cluster_analysis.get_cluster_resistance_LOR(presence_df, pheno_path)
lor_resistance_scores_dict =lor_resistance_scores.to_dict()['log_odds'] # -- we want it as cluster attributes later on

lor_resistance_scores

/home/rayane/miniconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,log_odds
Cluster 0,1.037304
Cluster 2,-1.125721
Cluster 4,1.899487
Cluster 6,-1.346896
Cluster 9,0.269622
...,...
Cluster 90932,0.269622
Cluster 90934,0.269622
Cluster 90936,0.269622
Cluster 90937,0.269622


In [2]:
lor_resistance_scores

,log_odds
Cluster 0,0.945557
Cluster 2,-0.571786
Cluster 4,0.264693
Cluster 6,-0.317071
Cluster 9,-1.125871
...,...
Cluster 90932,-0.428786
Cluster 90934,0.671504
Cluster 90936,-0.428786
Cluster 90937,0.671504


In [11]:
def generate_phenotype_df(pheno_path:str, presence_df:pd.DataFrame=None)->pd.DataFrame:
    '''
    takes in a path to a phenotype file to make a df out of it
    if a presence_df is provided, it will filter the pheno_df to only include samples that are in the presence_df

    NOW TAHT TEH PHENO FILES ARE PROCESSED TO INCLUDE ONLY PRESENT GENOME IDS, THIS ONLY TRANSFORMS INTO DF

    param:
    ----
        - pheno_path: str
        - presence_df: pd.DataFrame (optional)

    return:
    ----
        - pheno_df: pd.DataFrame
    '''
    pheno_df = pd.read_csv(pheno_path, index_col=0)

    # -- already processed, this will give errors
    # if presence_df is not None:
    #     samples_presence= presence_df.columns
    #     mask = pheno_df.index.astype(str).isin(samples_presence)
    #     pheno_df = pheno_df.loc[mask]

    return pheno_df

generate_phenotype_df(pheno_path, presence_df)

,SIR
genome_id,
562.23276,0
562.23275,1
562.23274,1
562.23273,1
562.23272,1
...,...
562.23824,1
562.23823,0
562.23822,1


In [10]:
os.path.exists(pheno_path)

pheno_df = network_construction.generate_phenotype_df(pheno_path, presence_df)
pheno_df.shape

(984, 1)

In [13]:
G = nx.read_graphml(f'results/Escherichia_coli_{drug}/{species}_{drug}_LOR.graphml')

G = cluster_analysis.set_node_attributes_by_cluster(cluster_info_df, G)
nx.set_node_attributes(G, lor_resistance_scores_dict, f'log_odds_{drug}_resistance')
G.nodes(data=True)

NodeDataView({'Cluster 4': {'gene_representative': '562.7699.peg.680', 'pan_gene_class': 'unique', 'product_name': 'T1SS secreted agglutinin RTX_37', 'frequency': 10, 'log_odds_amoxicillin_resistance': 0.0}, 'Cluster 66621': {'gene_representative': '562.23444.peg.4855', 'pan_gene_class': 'unique', 'product_name': 'Phage tail tip, host specificity protein J_93', 'frequency': 16, 'log_odds_amoxicillin_resistance': 0.0}, 'Cluster 75': {'gene_representative': '562.12960.peg.2226', 'pan_gene_class': 'unique', 'product_name': 'Invasin_2', 'frequency': 118, 'log_odds_amoxicillin_resistance': 0.0}, 'Cluster 74693': {'gene_representative': '562.16466.peg.3599', 'pan_gene_class': 'unique', 'product_name': 'Cytochrome b561 homolog 1_2', 'frequency': 30, 'log_odds_amoxicillin_resistance': 0.0}, 'Cluster 84': {'gene_representative': '562.23720.peg.1422', 'pan_gene_class': 'unique', 'product_name': 'iron aquisition yersiniabactin synthesis enzyme (Irp1,polyketide synthetase) @ Siderophore biosynthes

In [9]:

nx.write_graphml(G,f'results/Escherichia_coli_{drug}/{species}_{drug}_LOR_annotated.graphml')

NodeDataView({'Cluster 21': {'gene_representative': '562.23368.peg.1843', 'pan_gene_class': 'unique', 'product_name': "adherence and invasion outermembrane protein (Inv,enhances Peyer's patches colonization)_24", 'frequency': 17, 'log_odds_ampicillin_resistance': 1.2992829841302609}, 'Cluster 1156': {'gene_representative': '562.28841.peg.3170', 'pan_gene_class': 'unique', 'product_name': 'UPF0192 protein YfaS_17', 'frequency': 21, 'log_odds_ampicillin_resistance': 1.2992829841302609}, 'Cluster 8226': {'gene_representative': '562.30435.peg.2796', 'pan_gene_class': 'unique', 'product_name': 'Uric acid:proton symporter UacT_8', 'frequency': 15, 'log_odds_ampicillin_resistance': 1.2992829841302609}, 'Cluster 15358': {'gene_representative': '562.22434.peg.194', 'pan_gene_class': 'unique', 'product_name': 'Uncharacterized MFS-type transporter YdjK_2', 'frequency': 21, 'log_odds_ampicillin_resistance': 1.2992829841302609}, 'Cluster 18127': {'gene_representative': '562.22434.peg.3452', 'pan_ge

## issue #2